In [ ]:
TRAIN_PATH = '/content/drive/MyDrive/SIGNATE/DATASET/train.csv'
TEST_PATH = '/content/drive/MyDrive/SIGNATE/DATASET/test.csv'

In [ ]:
import keras.backend as K

def f1_score(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())

    return f1_val

In [ ]:
import pandas as pd

df_train = pd.read_csv(TRAIN_PATH)
df_test = pd.read_csv(TEST_PATH)

Y_train = df_train['state'].astype(int).values
test_id = df_test['id']

df_train = df_train.drop('state', axis=1)
df_train = df_train.drop('id', axis=1)
df_test = df_test.drop('id', axis=1)

df_train = df_train.drop('html_content', axis=1)
df_test = df_test.drop('html_content', axis=1)

df = pd.concat([df_train, df_test], axis=0)

In [ ]:
from sklearn.preprocessing import OneHotEncoder
import numpy as np

train_data = np.empty((len(df_train), 0), int)
test_data = np.empty((len(df_test), 0), int)

for column in df.columns:
  one_hot_encoder = OneHotEncoder(sparse=False)
  one_hot_encoder.fit(df[column].values.reshape(-1, 1))
  train_temp = one_hot_encoder.transform(df_train[column].values.reshape(-1, 1))
  test_temp = one_hot_encoder.transform(df_test[column].values.reshape(-1, 1))
  train_data = np.concatenate([train_data, train_temp], axis=1)
  test_data = np.concatenate([test_data, test_temp], axis=1)

print('train_data: {0}'.format(train_data.shape))
print('test_data: {0}'.format(test_data.shape))

train_data: (10545, 359)
test_data: (10544, 359)


|activation function|EPOCHS|optimizer|val_f1_score|Public Leaderboard|
|:--:|:--:|:--:|:--:|:--:|
|relu|200|Adagrad|0|0.7296|
|relu|500|Adagrad|0.7276|0|

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import RMSprop, Adagrad, SGD
from keras.losses import binary_crossentropy
from keras.metrics import binary_accuracy

model = Sequential()

model.add(Dense(100, activation='relu', input_shape=(359,)))
model.add(Dense(100, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# model.compile(optimizer=RMSprop(lr=0.001), loss=binary_crossentropy, metrics=[binary_accuracy, f1_score])
model.compile(optimizer=SGD(lr=1e-3), loss=binary_crossentropy, metrics=[binary_accuracy, f1_score])

EPOCHS=100
BATCH_SIZE=32

history = model.fit(train_data, Y_train, epochs=EPOCHS, batch_size=BATCH_SIZE, validation_split=0.2)

Epoch 1/100
264/264 [==============================] - 1s 3ms/step - loss: 0.6951 - binary_accuracy: 0.4841 - f1_score: 0.3785 - val_loss: 0.6933 - val_binary_accuracy: 0.4912 - val_f1_score: 0.4248
Epoch 2/100
264/264 [==============================] - 1s 2ms/step - loss: 0.6936 - binary_accuracy: 0.5057 - f1_score: 0.4413 - val_loss: 0.6928 - val_binary_accuracy: 0.5017 - val_f1_score: 0.4575
Epoch 3/100
264/264 [==============================] - 1s 2ms/step - loss: 0.6931 - binary_accuracy: 0.5131 - f1_score: 0.4848 - val_loss: 0.6923 - val_binary_accuracy: 0.5145 - val_f1_score: 0.4923
Epoch 4/100
264/264 [==============================] - 1s 2ms/step - loss: 0.6930 - binary_accuracy: 0.5033 - f1_score: 0.4845 - val_loss: 0.6918 - val_binary_accuracy: 0.5178 - val_f1_score: 0.5114
Epoch 5/100
264/264 [==============================] - 1s 2ms/step - loss: 0.6916 - binary_accuracy: 0.5236 - f1_score: 0.5206 - val_loss: 0.6913 - val_binary_accuracy: 0.5239 - val_f1_score: 0.5274
Epoch

In [ ]:
import matplotlib.pyplot as plt

acc = history.history['binary_accuracy']
val_acc = history.history['val_binary_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
f1 = history.history['f1_score']
val_f1 = history.history['val_f1_score']

epochs = range(1, EPOCHS+1)

plt.plot(epochs, acc, linestyle='solid', label='Training accuracy')
plt.plot(epochs, val_acc, linestyle='solid', color='orange', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()
plt.plot(epochs, loss, linestyle='solid', label='Training loss')
plt.plot(epochs, val_loss, linestyle='solid', color='orange', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.figure()
plt.plot(epochs, f1, linestyle='solid', label='Training f1 score')
plt.plot(epochs, val_f1, linestyle='solid', color='orange', label='Validation f1 score')
plt.title('Training and validation f1 score')
plt.legend()

plt.show()

In [ ]:
import numpy as np

predicted = model.predict(test_data)

df_sub = pd.concat([test_id, pd.DataFrame(predicted, columns=['predicted'])], axis=1)

print('df_sub shape: {0}'.format(df_sub.shape))

df_sub.to_csv('/content/drive/MyDrive/SIGNATE/DATASET/NN.csv', index=False, header=False)

df_sub.head()